## Backtester Problem Statement

#### Following are the reasons why we are building the library and what problems we have to solve while building it

1. To make backtesting faster. Not everyone researching an algorithm wants to write a framework to backtest.
2. Everyone in the team will be using the same metrics and formulas so less chance there is a bug affecting the results as well as faster bug discovery as everyone will be dogfooding the library.
3. Parallism can be leveraged to accelerate backtesting time decreasing parameter optimzation time as well as research time.
4. Better visualizations.

### Key Data Structures and Decisions

My design at this point of time is the following:

Backtester Class has following API:
- Backtest
- Trade History Visualization
- Metrics

Backtest method will take in OHLCV (+ other variables) data (only one atm) and two functions entry and monitor written by the user like :

`def backtest(self, entry, monitor):` where entry is the function that backtest runs to enter a position when backtesting and monitor is the function that monitors the position adding to and exiting the position based on the algorithm user provides.

In [6]:
import random
class Backtester:
    def __init__(self, data):
        self.data = data
        self.in_a_position = False
    def backtest(self, entry,monitor):
        for i in range(len(self.data)):
            if self.in_a_position:
                if monitor(self) == True:
                    self.in_a_position = False
                    print('we exited the trade')
            else:
                if entry(self) == True:
                    print('we entered a trade')
                    self.in_a_position = True

In [7]:
def entry(self):
    return random.randint(0,1)

In [8]:
def monitor(self):
    return random.randint(0,1)

In [9]:
backtester = Backtester([1,2,3,4,5])

In [16]:
backtester.backtest(entry,monitor)

we entered the trade
we monitored the trade
we entered the trade


My design for the backtest method looks as above and we leverage parallelism to backtest many parts of the data at the same time but the only problem will be a trade might not end before the segmented data ends resulting in skewed trading history at the end. To remedy this, I propose a re run to account for trades that do not end before the segmented data does. This will be demostrated in the future.

Trade History Visualization and Metrics are straight forward. The result of the backtest will be a dataframe like following:

In [18]:
import pandas as pd
pd.DataFrame(columns = ['Trade_Type','Allocation_Count', 'Entry_Timestamp', 'Exit_Timestamp', 'Allocation_History', 'Total_Allocation', 'Exit_Price', 'Trade_PNL','Aggregate_PNL', 'Stopped', 'Trade_Duration', 'Trade_amount'])

,Trade_Type,Allocation_Count,Entry_Timestamp,Exit_Timestamp,Allocation_History,Total_Allocation,Exit_Price,Trade_PNL,Aggregate_PNL,Stopped,Trade_Duration,Trade_amount


From which we make visualization maybe with Plot.ly to make a graph of the trade history allowing us to visualize where the trades are happening with Entry Timestamp and when they are ending with Exit_Timestamp and bunch of other variables

Metrics will be pretty straightforward as well. It will give us a different metrics and visualization based on the Trade History result we get from the backtest